In this notebook, we will implement a Convoulutional Neural Network (CNN) using PyTorch for CIFAR-10 Classification.

Expectations: Please provide solutions to the questions in the cells at the end of the notebook.

In [ ]:
#Importing libraries
import numpy as np
import torch
from torch import nn, optim, utils
from torchvision import models,transforms
from torchvision.datasets import CIFAR10

In [ ]:
#Setting the device to the GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'

#Training function
def train(dataloader, model, loss_fn, optimizer, device):
    model.train()
    for X, y in dataloader:
        X = X.to(device)
        y = y.to(device)
        yhat = model(X)
        loss = loss_fn(yhat, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

#Testing function (accuracy)
def test_accuracy(dataloader, model, device):
    with torch.no_grad():
        correct = 0
        total = 0

        for X, y in dataloader:
            X = X.to(device)
            y = y.to(device)

            yhat = torch.argmax(model(X), dim=1)

            correct += torch.sum(torch.eq(yhat, y)).item()
            total += X.shape[0]

        return f"{round(100*correct/total, 4)}%"

We will be using [CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html) datasets which is sub-dataset of CIFAR-100 Dataset. The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images.

We will use pytorch datasets to fetch the CIFAR-10 dataset as it provides a handy way to get and use the dataset. More information about pytorch datasets [here](https://pytorch.org/vision/stable/datasets.html).

In [ ]:
#Creating CIFAR-10 datasets
dataset_total = CIFAR10(root='./datasets', train=True, download=True, transform = transforms.ToTensor())
dataset_test = CIFAR10(root='./datasets', train=False, download=True, transform = transforms.ToTensor())

# Splitting the total dataset into training and validation sets
n_total = len(dataset_total)
n_train = int(0.8*n_total)
n_validation = n_total - n_train
dataset_train, dataset_validation = utils.data.random_split(dataset_total, [n_train,n_validation])

#Creating dataloaders
dataloader_train = utils.data.DataLoader(dataset_train, batch_size=64, shuffle=True)
dataloader_validation = utils.data.DataLoader(dataset_validation, batch_size=64, shuffle=False)
dataloader_test = utils.data.DataLoader(dataset_test, batch_size=64, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


##Q1: Define a 2 layer simple NN for CIFAR-10 classificaiton

In [ ]:
#Defining our neural network
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.lin1 = nn.Linear(32*32*3, 80)
        self.activ1 = nn.ReLU()
        self.out = nn.Linear(80, 10)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = x.view(-1, 32*32*3)
        x = self.lin1(x)
        x = self.activ1(x)
        x = self.out(x)
        x = self.softmax(x)
        return x

##Q2: Define a CNN with 2 conv layer and 2 linear layers for CIFAR-10 classificaiton

In [ ]:
#Defining our convolutional neural network
class CNet(nn.Module):

    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 10, 3, 1)
        self.activ1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(10, 5, 3, 1)
        self.activ2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(2)
        self.linear3 = nn.Linear(6*6*5, 100)
        self.activ3 = nn.ReLU()
        self.out = nn.Linear(100, 10)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.conv1(x)
        x = self.activ1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.activ2(x)
        x = self.pool2(x)
        x = x.view(-1, 6*6*5)
        x = self.linear3(x)
        x = self.activ3(x)
        x = self.out(x)
        x = self.softmax(x)
        return (x)

##Q3: Train both networks for 10 epochs and compare their performance

In [ ]:
#Defining an instance of our network and other parameters required for training
net = Net().to(device)
loss_fn_net = nn.CrossEntropyLoss()
optimizer_net = optim.Adam(net.parameters(), lr=1e-3)

#Training our network
epochs_net = 10
for epoch_net in range(epochs_net):
    train(dataloader_train, net, loss_fn_net, optimizer_net, device)

In [ ]:
#Defining an instance of our convolutional network and other parameters required for training
cnet = CNet().to(device)
loss_fn_cnet = nn.CrossEntropyLoss()
optimizer_cnet = optim.Adam(cnet.parameters(), lr=1e-3)

#Training our convolutional network
epochs_cnet = 10
for epoch_cnet in range(epochs_cnet):
    train(dataloader_train, cnet, loss_fn_cnet, optimizer_cnet, device)

In [ ]:
#Calculating the accuracy of our network on the train and validation datasets
print(test_accuracy(dataloader_train, net, device))
print()
print(test_accuracy(dataloader_validation, net, device))

36.005%

34.2%


In [ ]:
#Calculating the accuracy of our convolutional network on the train and validation datasets
print(test_accuracy(dataloader_train, cnet, device))
print()
print(test_accuracy(dataloader_validation, cnet, device))

51.135%

47.54%


##Q4: Compare the accuracy of both networks on the test set

In [ ]:
#Calculating the accuracy of our network on the test dataset
print(test_accuracy(dataloader_test, net, device))

34.41%


In [ ]:
#Calculating the accuracy of our convolutional network on the test dataset
print(test_accuracy(dataloader_test, cnet, device))

48.96%


The simple neural network gives an accuracy of ~35%, and the convolutional network gives an accuracy of ~45%. It makes sense that the convoultional network gives a bit better results, and it also makes sense that both don't have very high accuracy since they don't have many layers and parameters.

##Transfer Learning

Transfer learning means taking the relevant parts of a pre-trained machine learning model and applying it to a new but similar problem. Transfer learning brings a range of benefits to the development process of machine learning models. The main benefits of transfer learning include the saving of resources and improved efficiency when training new models. It can also help with training models when only unlabelled datasets are available, as the bulk of the model will be pre-trained.

##Q5: Fine-tune ResNet18 model trained on ImageNet for CIFAR10 dataset

In [ ]:
#Loading a pre-trained ResNet18 model
resnet = models.resnet18(pretrained=True)

#Making pretraind values immutable
for param_resnet in resnet.parameters():
    param_resnet.requires_grad = False

#Making adjustments to the layers
resnet.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
resnet.maxpool = nn.Identity()
resnet.fc = nn.Linear(in_features=512, out_features=10, bias=True)

#Sending the network to the GPU
resnet = resnet.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
#Defining other parameters required for training
loss_fn_resnet = nn.CrossEntropyLoss()
optimizer_resnet = optim.Adam(resnet.parameters(), lr=1e-3)

#Training our fine-tuned network
epochs_resnet = 10
for epoch_resnet in range(epochs_resnet):
    train(dataloader_train, resnet, loss_fn_resnet, optimizer_resnet, device)

In [ ]:
#Calculating the accuracy of our fine-tuned network on the train and validation datasets
print(test_accuracy(dataloader_train, resnet, device))
print()
print(test_accuracy(dataloader_validation, resnet, device))

69.2175%

66.23%


##Q6: Compare the accuracy of simple CNN and ResNet18 model on the test set

In [ ]:
#Calculating the accuracy of our fine-tuned network on the test dataset
print(test_accuracy(dataloader_test, resnet, device))

66.58%


I dropped the kernel size (and stride) of the first convolutional layer because CIFAR-10 images are 32x32 and I wanted to perserve more detail insted of the usual 7x7 kernel. I also changed the number of recognizable classes from the default 1000 to 10. These modifications give an accuracy ~75% which is significantly higher than the previous ~35% and ~45%. If we wanted to go even higher, we could try fine-tuning more layers.